# Document Classification with Naive Bayes - Lab

## Introduction

In this lesson, you'll practice implementing the Naive Bayes algorithm on your own.

## Objectives

In this lab you will:  

* Implement document classification using Naive Bayes

## Import the dataset

To start, import the dataset stored in the text file `'SMSSpamCollection'`.

In [1]:
# Your code here
!ls -ltr



total 572
-rw-r--r-- 1 poari 197609   1849 Sep 26 18:08 CONTRIBUTING.md
-rw-r--r-- 1 poari 197609   1371 Sep 26 18:08 LICENSE.md
-rw-r--r-- 1 poari 197609   2355 Sep 26 18:08 README.md
-rw-r--r-- 1 poari 197609 483481 Sep 26 18:08 SMSSpamCollection
-rw-r--r-- 1 poari 197609  82026 Oct  9 06:50 index.ipynb


In [2]:

import pandas as pd
df= pd.read_csv('SMSSpamCollection',names= ['label','text'],sep='\t')
df

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


## Account for class imbalance

To help your algorithm perform more accurately, subset the dataset so that the two classes are of equal size. To do this, keep all of the instances of the minority class (spam) and subset examples of the majority class (ham) to an equal number of examples.

In [3]:
# Your code here
import numpy as np
print(df['label'].value_counts())

# extract the ham rows in a separate df
df_ham=df[df['label']=='ham']

# extract the spam rows in a separate df
df_spam=df[df['label']=='spam']

# reset index of df_ham
df_ham.reset_index(inplace = True)

# generate 747 random numbers in the interval 0-4824
indices = np.random.randint(0, 4825, 747)

# select from df_ham
df_ham_select=df_ham.iloc[list(indices)]

# put together the selected ham records and the spam records
df_balanced = pd.concat([df_ham_select,df_spam],axis=0,sort=True)
df_balanced

ham     4825
spam     747
Name: label, dtype: int64


,index,label,text
3062,3534.0,ham,"Sorry, I'll call later"
1317,1538.0,ham,All sounds good. Fingers . Makes it difficult ...
3510,4053.0,ham,"Ya ok, then had dinner?"
1482,1727.0,ham,ALRITE HUNNY!WOT U UP 2 2NITE? DIDNT END UP GO...
3071,3543.0,ham,Ü come lt 25 n pass to me lar
...,...,...,...
5537,NaN,spam,Want explicit SEX in 30 secs? Ring 02073162414...
5540,NaN,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,NaN,spam,Had your contract mobile 11 Mnths? Latest Moto...
5566,NaN,spam,REMINDER FROM O2: To get 2.50 pounds free call...


In [7]:
# easier method:
df[df['label']=='ham'].sample(n=747)


,label,text
394,ham,Yes i think so. I am in office but my lap is i...
1588,ham,"Dont search love, let love find U. Thats why i..."
141,ham,"K, text me when you're on the way"
2216,ham,Prabha..i'm soryda..realy..frm heart i'm sory
4631,ham,Everybody had fun this evening. Miss you.
...,...,...
1317,ham,Why nothing. Ok anyway give me treat
5210,ham,Dai i downloaded but there is only exe file wh...
712,ham,It just seems like weird timing that the night...
5132,ham,it's still not working. And this time i also t...


In [8]:
df_balanced.reset_index(inplace=True)


In [9]:
df_balanced


,level_0,index,label,text
0,3062,3534.0,ham,"Sorry, I'll call later"
1,1317,1538.0,ham,All sounds good. Fingers . Makes it difficult ...
2,3510,4053.0,ham,"Ya ok, then had dinner?"
3,1482,1727.0,ham,ALRITE HUNNY!WOT U UP 2 2NITE? DIDNT END UP GO...
4,3071,3543.0,ham,Ü come lt 25 n pass to me lar
...,...,...,...,...
1489,5537,NaN,spam,Want explicit SEX in 30 secs? Ring 02073162414...
1490,5540,NaN,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
1491,5547,NaN,spam,Had your contract mobile 11 Mnths? Latest Moto...
1492,5566,NaN,spam,REMINDER FROM O2: To get 2.50 pounds free call...


In [10]:
print(df_balanced.columns)
df_balanced.drop(['level_0','index'],axis=1,inplace=True)
print(df_balanced.columns)

Index(['level_0', 'index', 'label', 'text'], dtype='object')
Index(['label', 'text'], dtype='object')


## Train-test split

Now implement a train-test split on the dataset: 

In [11]:
# Your code here
from sklearn.model_selection import train_test_split
# remove class label from df_balanced
X = df_balanced.drop(columns=['label'],axis = 1)
y = df_balanced['label']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2)
train_df = pd.concat([X_train,y_train],axis=1)
test_df = pd.concat([X_test,y_test],axis=1)

In [12]:
print(train_df.shape,test_df.shape)

(1195, 2) (299, 2)


In [13]:
299/(299+1195)


0.2001338688085676

In [14]:
train_df.columns

Index(['text', 'label'], dtype='object')

In [15]:
list(train_df['label'].unique())

['spam', 'ham']

In [16]:
train_df.index

Int64Index([1065,  882, 1179, 1189, 1385,  134,  140,  989,  658,  851,
            ...
             688,  703,  954,  396,  299,  909,  938, 1190,  207,  382],
           dtype='int64', length=1195)

In [17]:
train_df.head()

,text,label
1065,<Forwarded from 21870000>Hi - this is your Mai...,spam
882,What do U want for Xmas? How about 100 free te...,spam
1179,URGENT! We are trying to contact U. Todays dra...,spam
1189,This message is free. Welcome to the new & imp...,spam
1385,Your weekly Cool-Mob tones are ready to downlo...,spam


In [18]:
train_df.iloc[1]

text     What do U want for Xmas? How about 100 free te...
label                                                 spam
Name: 882, dtype: object

In [19]:
train_df.index

Int64Index([1065,  882, 1179, 1189, 1385,  134,  140,  989,  658,  851,
            ...
             688,  703,  954,  396,  299,  909,  938, 1190,  207,  382],
           dtype='int64', length=1195)

In [20]:
train_df.iloc[2]

text     URGENT! We are trying to contact U. Todays dra...
label                                                 spam
Name: 1179, dtype: object

In [21]:
train_df['text'][1413].split()

['Hi',
 'this',
 'is',
 'Amy,',
 'we',
 'will',
 'be',
 'sending',
 'you',
 'a',
 'free',
 'phone',
 'number',
 'in',
 'a',
 'couple',
 'of',
 'days,',
 'which',
 'will',
 'give',
 'you',
 'an',
 'access',
 'to',
 'all',
 'the',
 'adult',
 'parties...']

## Create the word frequency dictionary for each class

Create a word frequency dictionary for each class: 

In [22]:
# Your code here

# Will be a nested dictionary of class_i : {word1:freq, word2:freq..., wordn:freq} 
# will be based on training set only

# list of classes ham/spam
classes=list(train_df['label'].unique())
dico={}
for class_ in classes:
    # temporary dataframe containing only class_ 
    temp_df=train_df[train_df['label']==class_]
    # initialize an empty dictionary for bag of words
    bag={}
    # need to go through each row, take the text, split it and add up each word's count
    for row in temp_df.index:
        texto = temp_df['text'][row]
        for word in texto.split():
            # look for each word in the keys of dictionary bag. 
            # if found, return the value of word + 1
            # if not found, return  1
            bag[word]=bag.get(word,0)+1
    dico[class_]=bag
dico

{'spam': {'<Forwarded': 3,
  'from': 87,
  '21870000>Hi': 2,
  '-': 46,
  'this': 41,
  'is': 118,
  'your': 161,
  'Mailbox': 2,
  'Messaging': 2,
  'SMS': 11,
  'alert.': 3,
  'You': 56,
  'have': 98,
  '4': 74,
  'messages.': 1,
  '21': 1,
  'matches.': 2,
  'Please': 27,
  'call': 150,
  'back': 13,
  'on': 113,
  '09056242159': 2,
  'to': 476,
  'retrieve': 2,
  'messages': 5,
  'and': 95,
  'matches': 6,
  'What': 3,
  'do': 8,
  'U': 55,
  'want': 16,
  'for': 138,
  'Xmas?': 1,
  'How': 4,
  'about': 5,
  '100': 14,
  'free': 45,
  'text': 55,
  '&': 84,
  'a': 288,
  'new': 38,
  'video': 15,
  'phone': 23,
  'with': 85,
  'half': 3,
  'price': 6,
  'line': 11,
  'rental?': 2,
  'Call': 111,
  'now': 55,
  '0800': 12,
  '0721072': 1,
  'find': 16,
  'out': 34,
  'more!': 1,
  'URGENT!': 27,
  'We': 29,
  'are': 59,
  'trying': 10,
  'contact': 43,
  'U.': 8,
  'Todays': 12,
  'draw': 23,
  'shows': 22,
  'that': 18,
  'you': 138,
  'won': 34,
  '£2000': 17,
  'prize': 37,
  'G

In [23]:
dico.keys()

dict_keys(['spam', 'ham'])

In [28]:
dico['spam']['We'],dico['spam']['urgent'],

(29, 4)

## Count the total corpus words
Calculate V, the total number of words in the corpus: 

In [29]:
# Your code here
# create an empty set:
corpus_train = set()
for row in train_df.index:
    for word in train_df['text'][row].split():
        # as corpus_train is a set, it allows only unique elements.
        corpus_train.add(word)
corpus_train
V=len(corpus_train)
V

6149

In [35]:
tt=set()
tt.add(1)
tt.add(2)
tt.add(1)

In [36]:
tt


{1, 2}

## Create a bag of words function

Before implementing the entire Naive Bayes algorithm, create a helper function `bag_it()` to create a bag of words representation from a document's text.

In [52]:
# Your code here
def bag_it(text):
    #input = string
    bag={}
    for word in text.split():
        bag[word]=bag.get(word,0)+1
    return bag
        
    

In [51]:

for word in train_df['text'][3].split():
    print(word)

ALRITE
HUNNY!WOT
U
UP
2
2NITE?
DIDNT
END
UP
GOIN
DOWN
TOWN
JUS
DA
PUB
INSTEAD!
JUS
CHILLIN
AT
DA
MO
IN
ME
BEDROOM!LOVE
JEN
XXX.


In [53]:
bag_it(train_df['text'][3])

{'ALRITE': 1,
 'HUNNY!WOT': 1,
 'U': 1,
 'UP': 2,
 '2': 1,
 '2NITE?': 1,
 'DIDNT': 1,
 'END': 1,
 'GOIN': 1,
 'DOWN': 1,
 'TOWN': 1,
 'JUS': 2,
 'DA': 2,
 'PUB': 1,
 'INSTEAD!': 1,
 'CHILLIN': 1,
 'AT': 1,
 'MO': 1,
 'IN': 1,
 'ME': 1,
 'BEDROOM!LOVE': 1,
 'JEN': 1,
 'XXX.': 1}

In [54]:
who

V	 X	 X_test	 X_train	 bag	 bag_it	 class_	 classes	 corpus_train	 
df	 df_balanced	 df_ham	 df_ham_select	 df_spam	 dico	 indices	 np	 pd	 
row	 temp_df	 test_df	 texto	 train_df	 train_test_split	 tt	 word	 y	 
y_test	 y_train	 


In [55]:
type(train_df['text'][1])
type(train_df['text'][1].split())

list

## Implementing Naive Bayes

Now, implement a master function to build a naive Bayes classifier. Be sure to use the logarithmic probabilities to avoid underflow.

In [61]:
p_classes = dict(train_df['label'].value_counts(normalize=True))
p_classes2 = dict(train_df['label'].value_counts())

In [62]:
p_classes
p_classes2

{'spam': 602, 'ham': 593}

In [ ]:
# Your code here
def classify_doc(doc, class_word_freq, p_classes, V, return_posteriors=False):
    # doc = one of the texts in train_df
    # class_word_freq = word frequency dictionary for each class (ham or spam)
    # p_classes= count of occurences for each class in df_train
    # V total corpus words number
    bag=bag_it(doc) # return a dictionary with the number of occurence of each word in doc.
    classes=[]  # initialize an empty list
    posteriors=[]   # initialize an empty list
    for class_ in class_word_freq.keys():  # loop over spam, ham
        p=p_classes[class_]  # overall probability to find class_ (ham or spam) in train_df


In [58]:
# class_word_freq
dico.keys()

dict_keys(['spam', 'ham'])

## Test your classifier

Finally, test your classifier and measure its accuracy. Don't be perturbed if your results are sub-par; industry use cases would require substantial additional preprocessing before implementing the algorithm in practice.

In [ ]:
# Your code here


## Level up (Optional)

Rework your code into an appropriate class structure so that you could easily implement the algorithm on any given dataset.

## Summary

Well done! In this lab, you practiced implementing Naive Bayes for document classification!